In [8]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# install and download spaCy related modules
!pip install --upgrade spacy
!python -m spacy download en_core_web_lg
!pip install --upgrade tweepy

# Google Drive
from google.colab import drive

# Firebase/Firestore
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# BeautifulSoup
from bs4 import BeautifulSoup

# Tweepy
import tweepy

# general Python modules
import re
import json
import spacy
from spacy.language import Language
import requests
import datetime
from dateutil.relativedelta import relativedelta
from pprint import pprint

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 17 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 101 kB 5.8 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0


In [ ]:
# initialize Drive path
DRIVE_PATH = "/content/drive/MyDrive"

# open Firebase credentials
with open(DRIVE_PATH + "/IE/ie_course_2022_team03/credentials/firebase_credentials.json") as f:
  credential = json.load(f)
firebase_credential = credentials.Certificate(credential)

# create Firestore database instance
firebase_admin.initialize_app(firebase_credential)
db = firestore.client()
print("Stablished access to Firestore")

FileNotFoundError: ignored

In [ ]:
entities = {"berlin": [], "munich": []}

# Read urls for entities from json file
with open(DRIVE_PATH + "/IE/ie_course_2022_team03/notebooks/urls.json") as f:
  entities_data = json.load(f)

for i in entities.keys():
  entities[i] = entities_data[i]

print(entities)

FileNotFoundError: ignored

In [ ]:
def scrape_urls_data(entity_urls_list, total_saved_records, additional_urls):
  for root_url in entity_urls_list:

        # parse URLs in page using links parser
        response = requests.get(root_url)
        print(response.status_code)
        soup = BeautifulSoup(response.content, "html.parser")
        links = soup.find("div").findAll("a", href=True)
        #iterate URLs in the page
        for l in links:
          if total_saved_records == 500:
            return total_saved_records, additional_urls
          url = l["href"]
          if url.startswith("/en"):
            url = root_url + url[3:]
          elif url.startswith("/"):
            url = root_url + url
          
          try:
            response = requests.get(url)
          except:
            continue

          if response.status_code != 200:
            continue
          additional_urls.append(url)
          
          soup = BeautifulSoup(response.content, "html.parser")
          # check if h1 is present
          h1 = soup.find("h1")
          if h1 is not None and h1 != ' ':
            title = h1.text.strip()
          else:
            title = "_".join(url.split("/")[1:]).replace(".","_")[1:]
          # check if document has <p> elements, or at least 5
          p_els = soup.find_all("p")
          if p_els == None or len(p_els) <= 5:
            continue
          # collect paragraphs, if they are not empty
          html_container = [p.text for p in p_els if p.text.strip() != ""]
          # generate record ID
          id = re.search("https?://", url)
          id = url.replace(id.group(),"").replace("www","") # remove "https://" and "www"
          id = "".join(e for e in id if e.isalnum()) # remove all symbols and special chars
          # check uniqueness in database
          doc_ref = db.collection("texts").document(id)
          doc = doc_ref.get()
          if doc.exists:        
            print(f"-- Text record already exists: {url}")
          else:
            slots = {
              "id": id,
              "url": url,
              "title": title,
              "domain": re.search('https?://([A-Za-z_0-9.-]+).*', root_url).group(1),
              "text": html_container,
              "type": "web_scrape"
              }
            db.collection("texts").document(id).set(slots)
            with open(DRIVE_PATH + f"/IE/ie_course_2022_team03/retrieved_data/scrapped_data/{id}.json", "w", encoding = "utf-8") as f:
              json.dump(slots, f, ensure_ascii = False, indent = 4)
            print(total_saved_records, f"++ Saved text record: {title}")
            print(f"   {url}")
            total_saved_records += 1
  return total_saved_records, additional_urls

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
total_saved_records = 0

while True:

  additional_urls = []

  if total_saved_records >= 500:
    break

  if additional_urls != []:
    print("Additional urls", additional_urls)
    total_saved_records, additional_urls = scrape_urls_data(additional_urls, total_saved_records, additional_urls)
  else:
    for entity in entities.keys():
      entity_urls_list = entities[entity]
      print(entity, entity_urls_list)

      total_saved_records, additional_urls = scrape_urls_data(entity_urls_list, total_saved_records, additional_urls)
      

  print(f"{total_saved_records} text records saved!")

In [ ]:
# open Twitter credentials
with open(DRIVE_PATH + "/IE/ie_course_2022_team03/credentials/twitter_credentials.json") as f:
  twitter_credential = json.load(f)

# Initialize credentials to access Twitter API
auth = tweepy.OAuthHandler(
    twitter_credential["api_key"], twitter_credential["api_secret_key"])
auth.set_access_token(
    twitter_credential["access_token"], twitter_credential["access_token_secret"])
api = tweepy.API(auth)
client = tweepy.Client(bearer_token = twitter_credential["bearer_token"])

# client = tweepy.Client(consumer_key = twitter_credential["api_key"], consumer_secret = twitter_credential["api_secret_key"], access_token= twitter_credential["access_token"],access_token_secret= twitter_credential["access_token_secret"])

print("Stablished access to the Twitter API")

Stablished access to the Twitter API


In [ ]:
# Initialize selected fields to fetch from Twitter API
user_fields = ["created_at", "description", "id", "location", "name", 
               "pinned_tweet_id", "profile_image_url", "protected", 
               "public_metrics", "url", "username", "verified", "withheld"]
tweet_fields = ["attachments","author_id","conversation_id","created_at",
                "entities","id","in_reply_to_user_id","lang",
                "referenced_tweets","source","text","withheld"]

In [ ]:
twitter_user_name = "visitberlin"

user = client.get_user(username = twitter_user_name, user_fields = user_fields)
user = {
    "id": user.data.id,
    "username": user.data.username,
    "name": user.data.name,
    "verified": user.data.verified,
    "created_at": user.data.created_at,
    "description": user.data.description,
    "profile_image_url": user.data.profile_image_url.replace("normal", "400x400"),
    "pinned_tweet_id": user.data.pinned_tweet_id,
    "protected": user.data.protected,
    "public_metrics": user.data.public_metrics,
    "entities": user.data.entities,
    "url": user.data.url,
    "withheld": user.data.withheld
}
pprint(user)

{'created_at': datetime.datetime(2009, 1, 20, 20, 31, 46, tzinfo=datetime.timezone.utc),
 'description': 'Tips, events & news from #Berlin 🏙️ IG: visitberlin 🗨️ use '
                '#visitberlin 📸',
 'entities': None,
 'id': 19255509,
 'name': 'visitBerlin',
 'pinned_tweet_id': 1588487364128669697,
 'profile_image_url': 'https://pbs.twimg.com/profile_images/1569687001812684800/nCn4Wnh-_400x400.jpg',
 'protected': False,
 'public_metrics': {'followers_count': 92592,
                    'following_count': 928,
                    'listed_count': 841,
                    'tweet_count': 22665},
 'url': 'https://t.co/2y1sDMOd5C',
 'username': 'visitberlin',
 'verified': True,
 'withheld': None}


In [ ]:
#Get Twitter ID from Twitter user name
def get_twitter_id(screen_name):
  user = client.get_user(username = screen_name, user_fields = user_fields)
  return user.data.id

twitter_user_name = "MUC_Airport" #@param {type:"string"}
date_from = "2021-05-01" #@param {type:"date"}
date_to = "2021-05-31" #@param {type:"date"}
dtfrom_format = datetime.datetime.strptime(date_from, "%Y-%m-%d")
dtto_format = datetime.datetime.strptime(date_to, "%Y-%m-%d")
for i in range(24):

  dtfrom_format = dtfrom_format + relativedelta(months=1)
  dtto_format = dtto_format + relativedelta(months=1)
  date_from = str(dtfrom_format)
  date_to = str(dtto_format)
  start_time = date_from + "T00:00:00Z"
  end_time = date_to + "T23:59:00Z"

  next_page = None
  retrieved_tweets = []

  twitter_id = get_twitter_id(twitter_user_name)
  screen_name = twitter_user_name
  stop_pagination = False
  while stop_pagination == False:
    tweets = client.get_users_tweets(id = twitter_id,
                                    tweet_fields = tweet_fields,
                                    max_results = 10,
                                    start_time = start_time,
                                    end_time = end_time,
                                    pagination_token = next_page)
    if tweets.data is not None:
      if "next_token" in tweets.meta:
        next_page = tweets.meta["next_token"]
      else:
        next_page = None
        stop_pagination = True
      for tweet in tweets.data:
        data = {
          "id": tweet.id,
          "screen_name": screen_name,
          "author_id": tweet.author_id,
          "created_at": tweet.created_at.strftime("%Y-%m-%d %H:%M:%S"),
          "text": tweet.text,
          "conversation_id": tweet.conversation_id,
          "lang": tweet.lang
        }
        if tweet.entities is not None:
          data["entities"] = tweet.entities
        if tweet.in_reply_to_user_id is not None:
          data["in_reply_to_user_id"] = tweet.in_reply_to_user_id
        retrieved_tweets.append(data)
    else:
      if tweets.meta["result_count"] == 0 and next_page is not None:
        stop_pagination = True
        break
      else:
        print("Not tweets to extract with the parameters given")
        break
  if len(retrieved_tweets):
    # Save tweets to Drive in JSON format
    date_range = date_from.replace("-","")+"-"+date_to.replace("-","")
    file_name = f"tweets_@{twitter_user_name}_{date_range}"
    with open(f"/content/drive/MyDrive/IE/ie_course_2022_team03/retrieved_data/twitter_data/{file_name}.json", "w", encoding = "utf-8") as f:
      json.dump(retrieved_tweets, f, ensure_ascii = False, indent = 2)
    # Save tweets on Firestore      
    slots = {
      "id": file_name,
      "user_name": screen_name,
      "domain": "twitter.com",
      "type": "twitter",
      "title": f"Twitter @{screen_name}_{date_range}"
    }
    db.collection("tweets").document(file_name).set(slots)
    print(f'++ Completed retrieval of Twitter account "{screen_name}"')
  print("---")
  print(f"Retrieval date range: from {date_from} to {date_to}")
  print(f"Total tweets retrieved: {len(retrieved_tweets)}")

2021-06-01 00:00:00 2021-06-30 00:00:00 <class 'str'>
2021-07-01 00:00:00 2021-07-30 00:00:00 <class 'str'>
2021-08-01 00:00:00 2021-08-30 00:00:00 <class 'str'>
2021-09-01 00:00:00 2021-09-30 00:00:00 <class 'str'>
2021-10-01 00:00:00 2021-10-30 00:00:00 <class 'str'>
2021-11-01 00:00:00 2021-11-30 00:00:00 <class 'str'>
2021-12-01 00:00:00 2021-12-30 00:00:00 <class 'str'>
2022-01-01 00:00:00 2022-01-30 00:00:00 <class 'str'>
2022-02-01 00:00:00 2022-02-28 00:00:00 <class 'str'>
2022-03-01 00:00:00 2022-03-28 00:00:00 <class 'str'>
2022-04-01 00:00:00 2022-04-28 00:00:00 <class 'str'>
2022-05-01 00:00:00 2022-05-28 00:00:00 <class 'str'>
2022-06-01 00:00:00 2022-06-28 00:00:00 <class 'str'>
2022-07-01 00:00:00 2022-07-28 00:00:00 <class 'str'>
2022-08-01 00:00:00 2022-08-28 00:00:00 <class 'str'>
2022-09-01 00:00:00 2022-09-28 00:00:00 <class 'str'>
2022-10-01 00:00:00 2022-10-28 00:00:00 <class 'str'>
2022-11-01 00:00:00 2022-11-28 00:00:00 <class 'str'>
2022-12-01 00:00:00 2022-12-

In [2]:
def clean_document(text):
  if text.find("’"):
    text = text.replace("’", "'")
  text = re.sub(r"won\'t", "will not", text)
  text = re.sub(r"can\'t", "can not", text)
  text = re.sub(r"in\'", "ing", text)
  text = re.sub(r"\'em", "them", text)
  text = re.sub(r"n\'t", " not", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'s", " is", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'t", " not", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'m", " am", text)
  text = " ".join(text.split())
  return text

In [10]:
list_files = !ls /content/drive/MyDrive/IE/ie_course_2022_team03/retrieved_data/scrapped_data

In [4]:
!python -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [11]:
nlp = spacy.load("en_core_web_md")

nlp.add_pipe("entityLinker", last=True)

qid, unqiue_qids = [], []

DRIVE_PATH = "/content/drive/MyDrive"

for file in list_files:
  if len(unqiue_qids) > 600:
    break
  with open(DRIVE_PATH + f"/IE/ie_course_2022_team03/retrieved_data/scrapped_data/{file}") as f:
    file_data = json.load(f)
  textual_data =  " ".join(file_data['text'])
  processed_data = clean_document(textual_data)
  doc = nlp(processed_data)
  all_linked_entities = doc._.linkedEntities
  for i in all_linked_entities:
    qid.append("Q"+str(i.get_id()))
  unqiue_qids = [*set(qid)]
print(unqiue_qids)

['Q2603675', 'Q621206', 'Q842119', 'Q11255', 'Q34302', 'Q20136', 'Q4686406', 'Q15', 'Q1653073', 'Q4678319', 'Q333173', 'Q43417', 'Q210756', 'Q35872', 'Q1087850', 'Q740437', 'Q11184', 'Q2094083', 'Q2757977', 'Q848532', 'Q55064534', 'Q9264228', 'Q2435869', 'Q1014852', 'Q2693730', 'Q9081', 'Q840343', 'Q11195', 'Q28877', 'Q690182', 'Q1753003', 'Q720314', 'Q468565', 'Q778505', 'Q30093000', 'Q222', 'Q25235', 'Q18028877', 'Q178024', 'Q1144560', 'Q536168', 'Q812285', 'Q847337', 'Q893', 'Q191768', 'Q7015', 'Q9301841', 'Q145', 'Q61509', 'Q1621107', 'Q47528', 'Q10998', 'Q57838809', 'Q84', 'Q183252', 'Q6501349', 'Q11851202', 'Q2085515', 'Q1135620', 'Q33767', 'Q862867', 'Q182832', 'Q1551523', 'Q634', 'Q2326384', 'Q40276', 'Q3278344', 'Q334445', 'Q48282', 'Q2596997', 'Q6460735', 'Q3044', 'Q482891', 'Q29049', 'Q214796', 'Q44', 'Q187956', 'Q15713092', 'Q17103040', 'Q7691853', 'Q605704', 'Q23220', 'Q3336843', 'Q762', 'Q2920963', 'Q124794', 'Q78994', 'Q1052767', 'Q192110', 'Q37503541', 'Q15276466', 'Q36

In [12]:
print(len(unqiue_qids))
print(len(qid))

640
1288


In [13]:
entity_list = []
for id in unqiue_qids:
  try:
    root_url = "https://www.wikidata.org/wiki/"+id
    response= requests.get(root_url)
    d={}
    soup = BeautifulSoup(response.content,"html.parser")
    item = soup.find("div",{"id": "content"})
    qid = item.find("span",{"class":"wikibase-title-id"})
    d["qid"] = qid.get_text().replace("(","").replace(")","")
    d["name"] = item.find("span",{"class":"wikibase-title-label"}).get_text()
    if(item.find("div",{"class":"wikibase-aliasesview "}) == None):
      d["aliases"] = ""
    else:
      d["aliases"]= item.find("div",{"class":"wikibase-aliasesview "}).get_text(separator="\n",strip=True).split("\n")

    d["description"] = str(item.find("div",{"class":"wikibase-entitytermsview-heading-description "}).get_text())
    d["label"] = ' '.join([i.label_ for i in nlp(d["name"]).ents])
    json_object = json.dumps(d, indent = 4) 
    entity_list.append(d)
    print("Data stored with Qid ", id)
  except Exception as e:
    print(e)

print(entity_list)
print(len(entity_list))
name_list = list(i['name'] for i in entity_list)
alias_list = list(i['aliases'] for i in entity_list)
alias_list = [item for sublist in alias_list for item in sublist]
gazetteers_list=name_list+alias_list
print(gazetteers_list)

Data stored with Qid  Q2603675
Data stored with Qid  Q621206
Data stored with Qid  Q842119
Data stored with Qid  Q11255
Data stored with Qid  Q34302
Data stored with Qid  Q20136
Data stored with Qid  Q4686406
Data stored with Qid  Q15
Data stored with Qid  Q1653073
Data stored with Qid  Q4678319
Data stored with Qid  Q333173
Data stored with Qid  Q43417
Data stored with Qid  Q210756
Data stored with Qid  Q35872
Data stored with Qid  Q1087850
Data stored with Qid  Q740437
Data stored with Qid  Q11184
Data stored with Qid  Q2094083
Data stored with Qid  Q2757977
Data stored with Qid  Q848532
Data stored with Qid  Q55064534
Data stored with Qid  Q9264228
Data stored with Qid  Q2435869
Data stored with Qid  Q1014852
Data stored with Qid  Q2693730
Data stored with Qid  Q9081
Data stored with Qid  Q840343
Data stored with Qid  Q11195
Data stored with Qid  Q28877
Data stored with Qid  Q690182
Data stored with Qid  Q1753003
Data stored with Qid  Q720314
Data stored with Qid  Q468565
Data store

In [14]:
with open(f'{DRIVE_PATH}/IE/ie_course_2022_team03/assets/entities.json', 'w') as file:
  json.dump(entity_list,file,indent=4)
with open(f'{DRIVE_PATH}/IE/ie_course_2022_team03/assets/entities_gazetteers.json', 'w') as file:
  json.dump(gazetteers_list,file,indent=4)

In [ ]:
# retrieve list of entities gazetteers
with open(DRIVE_PATH + "/IE/ie_course_2022_team03/assets/entities_gazetteers.json") as f:
  entities_gazetteers_list = json.load(f)
  print(f"Retrieved entities gazetteers list")
  
# retrieve ignore list of entities gazetteers
with open(DRIVE_PATH + "/IE/ie_course_2022_team03/assets/entities_gazetteers_ignore_list.json") as f:
  entities_gazetteers_ignore_list = json.load(f)
  print(f"Retrieved entities gazetteers' ignore list")

# retrieve entities
with open(DRIVE_PATH + "/IE/ie_course_2022_team03/assets/entities.json") as f:
  entities = json.load(f)
  print(f"Retrieved entities")

Retrieved entities gazetteers list
Retrieved entities gazetteers' ignore list
Retrieved entities


In [ ]:
""" Custom pipeline Component: entities gazetteer function """

@Language.component("entities_gazetteer")
def entities_gazetteer(doc):
  # set up and extend structure of default span object
  span = Span(doc, 0, 0, "")
  span.set_extension("qid", default=None, force=True)
  span.set_extension("label", default=None, force=True)
  span.set_extension("wd_name", default=None, force=True)

  # identify matches of the gazetteers contained in Doc object (text)
  matches = matcher(doc)
  # convert matches to Span objects
  spans = [doc[start:end] for _, start, end in matches]
  # filter overlaping matches (Span objs) to keep gazetteers uniqueness
  filtered_matches = spacy.util.filter_spans(spans)

  # loop unique matches of gazetteers
  for match in filtered_matches:
    # skip if matched gazetter is in ignore list
    if match.text in entities_gazetteers_ignore_list:
      print(f"-- Skipped '{match}' due it's in ignore list!")
      continue
    # find matched gazetters in issues dictionary to get entities' Wikidata info
    # usually only one entity is found, but some gazetteer finds more than one
    matched_entities = [i for i in entities if match.text == i["name"] or match.text in i["aliases"]]
    if len(matched_entities):
      entity = Span(doc, match.start, match.end, label=matched_entities[0]["label"])

      # set attributes
      if len(matched_entities) == 1:
        entity._.label = matched_entities[0]["label"]
      elif len(matched_entities) > 1:
        entity._.label = [e["label"] for e in matched_entities]

      entity.set_extension("qid", default=None, force=True)
      if len(matched_entities) == 1:
        entity._.qid = matched_entities[0]["qid"]
      elif len(matched_entities) > 1:
        entity._.qid = [e["qid"] for e in matched_entities]

      entity.set_extension("wd_name", default=None, force=True)
      if len(matched_entities) == 1:
        entity._.wd_name = matched_entities[0]["name"]
      elif len(matched_entities) > 1:
        entity._.wd_name = [e["name"] for e in matched_entities]

      # modify the provided entity spans, leaving the rest unmodified
      doc.set_ents([entity], default="unmodified")

  return doc

# create pipeline loaded with a pretrained statistical model (English/lg)
nlp = spacy.load("en_core_web_lg", exclude=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])
nlp.add_pipe("sentencizer")

# add custom component to pipeline
nlp.add_pipe("entities_gazetteer", last=True)

# initialize spaCY phrase matcher (rule-based)
matcher = PhraseMatcher(nlp.vocab, None)

# load gazetteers (issues) as matcher patterns
patterns = [nlp.make_doc(gazetteer) for gazetteer in entities_gazetteers_list]
matcher.add("gazetteers", patterns)

# see pipeline components
print(nlp.pipe_names)

# analize pipeline
pprint(nlp.analyze_pipes(pretty=True))

['ner', 'sentencizer', 'entities_gazetteer']

============================= Pipeline Overview =============================

#   Component            Assigns               Requires   Scores          Retokenizes
-   ------------------   -------------------   --------   -------------   -----------
0   ner                  doc.ents                         ents_f          False      
                         token.ent_iob                    ents_p                     
                         token.ent_type                   ents_r                     
                                                          ents_per_type              
                                                                                     
1   sentencizer          token.is_sent_start              sents_f         False      
                         doc.sents                        sents_p                    
                                                          sents_r                    
               

In [ ]:
# initialize main container of text
main_text_container = []
# main_text_woc_container = []
text = text_rec["texts"]

# split text in paragraphs
for index, paragraph in enumerate(text):
  # split paragraph in sentences
  sentences = [sent.text for sent in nlp(paragraph).sents]

  # process sentences individually in a stream (multi-thread)
  for doc in nlp.pipe(sentences, batch_size=50):
    sent = doc.text  # sentence
    chosen_sentence = [sent, {"entities": []}]
    for ent in doc.ents:
      if ent._.qid:
        chosen_sentence[1]["entities"].append([ent.start_char, ent.end_char, ent._.qid, ent.text, ent.label_, ent._.label, ent._.wd_name])
    if len(chosen_sentence[1]["entities"]):
      main_text_container.append(chosen_sentence)
      print()
      print(f"++ {sent}")
      for ent in doc.ents:
        if ent._.qid:
          print(f"    {ent.text, ent.start_char, ent.end_char, ent.label_, ent._.label, ent._.qid, ent._.wd_name}")
      print()
      
# save record in JSON file
if len(main_text_container):
  with open(DRIVE_PATH + "/ie_course/output/extracted_sentences.json", "w", encoding = "utf-8") as f:
    json.dump(main_text_container, f, ensure_ascii = False, indent = 2)
    print()
    print(f"Saved {len(main_text_container)} extracted_sentences")